In [9]:
#nltk.download()

In [1]:
import urllib
from bs4 import BeautifulSoup
import itertools
import os
import codecs
import csv
import glob
import ssl
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from nltk.corpus import stopwords

In [5]:
def briefing_extractor_list(urllist):

    for url in urllist:
        
        #url = "https://www.whitehouse.gov/the-press-office/2017/01/21/statement-press-secretary-sean-spicer"

        #accommodate some sites that do not have valid https verification running.
        #WARNING: BE AWARE OF WHAT SITE YOU ARE PULLING INFORMATION FROM

        context = ssl._create_unverified_context()
        html = urllib.urlopen(url, context=context).read()

        soup = BeautifulSoup(html)
        
        subname = "briefings"
        
        filename = os.path.join("EDAVtest_" + url[url.index(subname):]+ "." + "txt")
        filename = filename.replace('/','_')

        #print filename
        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.body.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk).encode('ascii', 'ignore')

        #identify header/footer, extract text between
        header = 'FLEISCHER:'
         
        footer = 'END'
        
        body = text[text.index(header):text.index(footer)]

        #print body or write to file

        #print body
        
        with open(filename, "w") as f:
                f.write(body)

In [6]:
urllist = ("https://georgewbush-whitehouse.archives.gov/news/briefings/20010125.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010125-1.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010126.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010129.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010130.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010131.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010201.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010202.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010205.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010206.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010207.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010208.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010209.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010215.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010226.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010227.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010302.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010305.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010307.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010313.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010320.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010322.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010323.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010326.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010327.html", 
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010328.html", 
           #"https://georgewbush-whitehouse.archives.gov/news/briefings/20010330.html", 
           #note.. even though labeled as Fleischer, was actually Scott McClellan
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010330-1.html",
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010404.html",
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010405.html",
           "https://georgewbush-whitehouse.archives.gov/news/briefings/20010406.html"
)

In [7]:
briefing_extractor_list(urllist)

In [8]:
#Append all individual briefings to single text file, fix encoding for subsequent NLTK steps

text = '*EDAVtest*.txt'

read_files = glob.glob(text)

filename = 'EDAVtest_compiled.txt'
with open(filename, "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            
            #outfile.write(infile.read())
            in_byte_string = infile.read()
            unicode_string = in_byte_string.decode('UTF-8')
            out_byte_string = unicode_string.encode('ASCII', 'ignore')

            outfile.write(out_byte_string)


In [9]:
def produce_frequency_distributions(text_file):

        
        #establish stop words
        stop_words = set(stopwords.words('english'))

        #some updates based on manual txt analysis
        stop_words.update(['mr', 'fleischer','mcclellan', '.', ',', '?', '!', ';', '(', ')', '[', ']', '{', '}', '-', '--', ':', 'q', "'", '#', '.)'])

        #remove stop words from text file
        with open(text_file, "r") as f1:
            raw = f1.read()
            #raw_nostop = [i for i in raw.lower().split() if i not in stop_words]
            raw_nostop = [i.lower() for i in wordpunct_tokenize(raw) if i.lower() not in stop_words]
            str_nostop = " ".join(raw_nostop)

        #tokenize str extracted from text file
        tokens = nltk.word_tokenize(str_nostop)

        #Create n-grams
        bgs = nltk.bigrams(tokens)
        tgs = nltk.trigrams(tokens)

        
            
        #compute frequency distributions: unigrams, bigrams, trigrams
        bigrams = nltk.FreqDist(bgs)
        bi = "bigrams"
        trigrams = nltk.FreqDist(tgs)
        tri = "trigrams"
        unigrams = nltk.FreqDist(tokens)
        uni = "unigrams"
        gramslist = [(unigrams,uni), (bigrams,bi), (trigrams,tri)]
        
        for grams in gramslist:
            
            #write to csv: note- change file name and writer.writerows arg as needed. also, fuck trump.
            with open('%s%s.csv' % (text_file[:text_file.index("_")], grams[1]), "wb") as fuck_trump:
                writer = csv.writer(fuck_trump, quoting=csv.QUOTE_ALL)
                writer.writerows(grams[0].items())

            #for k,v in fdist.items():
                #print k,v

In [42]:
produce_frequency_distributions("EDAVtest_compiled.txt")